In [84]:
from collections import Counter
import numpy as np
import komm
import struct


In [85]:

def contar_caracteres(arquivo):
    contador = {}
    pmf = {}
    
    # Ler o arquivo e contar caracteres
    with open(arquivo, 'rb') as f:  # Mudado para 'rb'
        texto = f.read().decode('utf-8')  # Decodifica após ler os bytes
        total_chars = len(texto)
        
        for char in texto:
            contador[char] = contador.get(char, 0) + 1
    
    # Calcular PMF
    for char, freq in contador.items():
        pmf[char] = freq / total_chars
    
    return contador, pmf, texto

In [86]:
def contar_bytes(arquivo):
    """
    Conta o número total de bytes no arquivo
    Retorna um dicionário com o tamanho em diferentes unidades
    """
    try:
        with open(arquivo, 'rb') as f:
            conteudo = f.read()
            tamanho_bytes = len(conteudo)
            
        return {
            'bytes': tamanho_bytes,
            'KB': round(tamanho_bytes / 1024, 2),
            'MB': round(tamanho_bytes / (1024 * 1024), 2)
        }
    except FileNotFoundError:
        return f"Erro: Arquivo '{arquivo}' não encontrado."
    except Exception as e:
        return f"Erro ao ler arquivo: {str(e)}"


In [87]:
arquivo = 'alice.txt'

# Contar caracteres e obter texto
contador, pmf, texto_original = contar_caracteres(arquivo)


# Exibir contagem de caracteres em ordem decrescente
print("Contagem de caracteres:")
# Ordenar por quantidade (valor) em ordem decrescente
for char, quantidade in sorted(contador.items(), key=lambda x: x[1], reverse=True):
    print(f'{char}: {quantidade}')

Contagem de caracteres:
 : 24633
e: 13552
t: 10345
a: 8239
o: 8073
h: 7176
n: 6955
i: 6856
s: 6361
r: 5376
d: 4779
l: 4681
u: 3461
: 3383

: 3383
w: 2468
g: 2462
,: 2425
c: 2278
y: 2193
f: 1938
m: 1934
p: 1469
b: 1401
“: 1118
”: 1114
k: 1083
.: 1001
v: 837
’: 706
I: 689
A: 589
!: 451
_: 440
T: 393
—: 263
:: 233
H: 223
W: 210
?: 204
;: 193
M: 176
D: 162
S: 154
-: 146
C: 145
x: 144
j: 138
q: 129
R: 98
O: 96
B: 86
Q: 83
E: 80
K: 78
N: 77
z: 77
P: 76
G: 76
*: 72
Y: 72
F: 67
(: 56
): 56
L: 54
‘: 46
U: 18
V: 16
J: 10
X: 8
1: 4
[: 3
]: 3
3: 1
0: 1
Z: 1
ù: 1


In [88]:
# Ordenar a PMF em ordem decrescente
sorted_pmf = sorted(pmf.items(), key=lambda item: item[1], reverse=True)

# Criar código de Huffman
probs = [prob for _, prob in sorted_pmf]



print("\nPMF:")
for char, prob in sorted_pmf:
    print(f'{char}: {prob:.6f}')
    
huffman = komm.HuffmanCode(probs)

dms = komm.DiscreteMemorylessSource(probs)
print("A entropia da distribuição de frequências dos caracteres do livro:", dms.entropy())
print("Comprimento médio do código de Huffman obtido:", huffman.rate(probs))


PMF:
 : 0.166352
e: 0.091519
t: 0.069862
a: 0.055640
o: 0.054519
h: 0.048461
n: 0.046968
i: 0.046300
s: 0.042957
r: 0.036305
d: 0.032274
l: 0.031612
u: 0.023373
: 0.022846

: 0.022846
w: 0.016667
g: 0.016626
,: 0.016377
c: 0.015384
y: 0.014810
f: 0.013088
m: 0.013061
p: 0.009920
b: 0.009461
“: 0.007550
”: 0.007523
k: 0.007314
.: 0.006760
v: 0.005652
’: 0.004768
I: 0.004653
A: 0.003978
!: 0.003046
_: 0.002971
T: 0.002654
—: 0.001776
:: 0.001573
H: 0.001506
W: 0.001418
?: 0.001378
;: 0.001303
M: 0.001189
D: 0.001094
S: 0.001040
-: 0.000986
C: 0.000979
x: 0.000972
j: 0.000932
q: 0.000871
R: 0.000662
O: 0.000648
B: 0.000581
Q: 0.000561
E: 0.000540
K: 0.000527
N: 0.000520
z: 0.000520
P: 0.000513
G: 0.000513
*: 0.000486
Y: 0.000486
F: 0.000452
(: 0.000378
): 0.000378
L: 0.000365
‘: 0.000311
U: 0.000122
V: 0.000108
J: 0.000068
X: 0.000054
1: 0.000027
[: 0.000020
]: 0.000020
3: 0.000007
0: 0.000007
Z: 0.000007
ù: 0.000007
A entropia da distribuição de frequências dos caracteres do livro: 4.62

In [89]:
def comprimir_arquivo(texto, codigos, arquivo_saida):
    """Codifica o texto e salva como arquivo binário com a tabela de códigos embutida"""
    # Codificar o texto
    texto_codificado = ''.join(codigos[char] for char in texto)
    
    # Adicionar padding
    padding = 8 - (len(texto_codificado) % 8)
    if padding != 8:
        texto_codificado += '0' * padding
    
    # Converter para bytes
    bytes_array = bytearray()
    for i in range(0, len(texto_codificado), 8):
        byte = texto_codificado[i:i+8]
        bytes_array.append(int(byte, 2))
    
    # Salvar arquivo comprimido
    with open(arquivo_saida, 'wb') as f:
        # Salvar padding
        f.write(bytes([padding]))
        
        # Salvar tabela de códigos
        # Primeiro salvamos o número de entradas na tabela
        f.write(struct.pack('I', len(codigos)))
        
        # Salvar cada par (caractere, código)
        for char, code in codigos.items():
            # Salvar o caractere
            char_bytes = char.encode('utf-8')
            f.write(struct.pack('B', len(char_bytes)))
            f.write(char_bytes)
            
            # Salvar o código
            code_len = len(code)
            f.write(struct.pack('B', code_len))
            f.write(int(code, 2).to_bytes((code_len + 7) // 8, byteorder='big'))
        
        # Salvar dados comprimidos
        f.write(bytes_array)
    
    return len(bytes_array)

In [90]:
def descomprimir_arquivo(arquivo_entrada, arquivo_saida):
    """Lê o arquivo binário e reconstrói o texto original usando a tabela de códigos embutida"""
    with open(arquivo_entrada, 'rb') as f:
        # Ler padding
        padding = int.from_bytes(f.read(1), byteorder='big')
        
        # Ler tabela de códigos
        num_codes = struct.unpack('I', f.read(4))[0]
        codigos = {}
        
        # Reconstruir tabela de códigos
        for _ in range(num_codes):
            # Ler caractere
            char_size = struct.unpack('B', f.read(1))[0]
            char = f.read(char_size).decode('utf-8')
            
            # Ler código
            code_len = struct.unpack('B', f.read(1))[0]
            code_bytes = f.read((code_len + 7) // 8)
            code = format(int.from_bytes(code_bytes, byteorder='big'), f'0{code_len}b')
            
            codigos[char] = code
        
        # Ler dados comprimidos
        dados_comprimidos = f.read()
    
    # Converter para bits
    bits = ''.join(format(byte, '08b') for byte in dados_comprimidos)
    if padding != 8:
        bits = bits[:-padding]
    
    # Decodificar
    codigos_reversos = {v: k for k, v in codigos.items()}
    codigo_atual = ''
    texto_decodificado = []
    
    for bit in bits:
        codigo_atual += bit
        if codigo_atual in codigos_reversos:
            texto_decodificado.append(codigos_reversos[codigo_atual])
            codigo_atual = ''
    
    # Salvar arquivo descomprimido
    with open(arquivo_saida, 'w', encoding='utf-8') as f:
        f.write(''.join(texto_decodificado))

In [91]:
#função apenas para juntar os bits gerados pela função do komm
def criar_codigo_huffman(pmf):
    """Cria os códigos de Huffman usando diretamente as probabilidades"""
    # Converter o dicionário PMF em uma lista ordenada de probabilidades
    sorted_pmf = sorted(pmf.items(), key=lambda x: x[1], reverse=True)
    caracteres = [char for char, _ in sorted_pmf]
    probabilidades = [prob for _, prob in sorted_pmf]
    
    # Criar o código Huffman
    huffman = komm.HuffmanCode(probabilidades)
    
    # Converter os codewords (que são tuplas) em strings binárias
    codigos = {}
    for i, char in enumerate(caracteres):
        codeword = huffman.codewords[i]
        codigo_binario = ''.join(str(bit) for bit in codeword)
        codigos[char] = codigo_binario
    
    return huffman, codigos


In [93]:
# Criar código de Huffman
huffman, codigos = criar_codigo_huffman(pmf)

# Comprimir arquivo
arquivo_comprimido = 'alice.bin'
tamanho_comprimido = comprimir_arquivo(texto_original, codigos, arquivo_comprimido)

# Descomprimir arquivo
arquivo_descomprimido = 'alice_descomprimido.txt'
descomprimir_arquivo(arquivo_comprimido, arquivo_descomprimido)

# Calcular e exibir estatísticas finais
tamanho_original = contar_bytes(arquivo)['bytes']
tamanho_descomprimido = contar_bytes(arquivo_descomprimido)['bytes']
taxa_compressao = (1 - (tamanho_comprimido / tamanho_original)) * 100

print("\nEstatísticas de compressão:")
print(f"Tamanho original: {tamanho_original:,} bytes")
print(f"Tamanho comprimido: {tamanho_comprimido:,} bytes")
print(f"Tamanho descomprimido: {tamanho_descomprimido:,} bytes")
print(f"Taxa de compressão: {taxa_compressao:.2f}%")


Estatísticas de compressão:
Tamanho original: 154,573 bytes
Tamanho comprimido: 86,278 bytes
Tamanho descomprimido: 154,573 bytes
Taxa de compressão: 44.18%
